<a href="https://colab.research.google.com/github/rajcm17/FinalProject/blob/dataset/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train = pd.read_csv('train.tsv', sep = '\t', header=None)
test = pd.read_csv('test.tsv', sep = '\t', header=None)
valid = pd.read_csv('valid.tsv', sep = '\t', header=None)

columns = ['id','label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
           'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts',
           'pants_fire_counts','context']
train.columns = columns
test.columns = columns
valid.columns = columns
train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'test.tsv'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.tsv to test.tsv
Saving train.tsv to train (1).tsv
Saving valid.tsv to valid.tsv


In [ ]:
import os
os.listdir()

['.config', 'train.tsv', 'sample_data']

In [ ]:
import pandas as pd
train = pd.read_csv('train.tsv', sep = '\t', header=None)
test = pd.read_csv('test.tsv', sep = '\t', header=None)
valid = pd.read_csv('valid.tsv', sep = '\t', header=None)

columns = ['id','label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
           'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts',
           'pants_fire_counts','context']
train.columns = columns
test.columns = columns
valid.columns = columns
train.head()


,id,label,statement,subject,speaker,job,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_fire_counts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [ ]:
# Binary: fake=0, real=1
fake_labels = ['pants-fire', 'false', 'barely-true']
real_labels = ['half-true', 'mostly-true', 'true']

def map_label(label):
    if label in fake_labels:
        return 0
    else:
        return 1

train['label_bin'] = train['label'].apply(map_label)
test['label_bin'] = test['label'].apply(map_label)
valid['label_bin'] = valid['label'].apply(map_label)

In [ ]:
import re
import string

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

train['statement_clean'] = train['statement'].apply(clean_text)
test['statement_clean'] = test['statement'].apply(clean_text)
valid['statement_clean'] = valid['statement'].apply(clean_text)

In [ ]:
X_train = train['statement_clean']
y_train = train['label_bin']

X_test = test['statement_clean']
y_test = test['label_bin']

X_valid = valid['statement_clean']
y_valid = valid['label_bin']

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode_texts(texts, max_len=128):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors="tf"
    )

train_encodings = encode_texts(X_train)
test_encodings = encode_texts(X_test)
valid_encodings = encode_texts(X_valid)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
statement = "The COVID-19 vaccine is safe and effective."
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokens = tokenizer.tokenize(statement)
token_ids = tokenizer.encode(statement)

print("Tokens:",tokens)
print("Token IDs:",token_ids)

Tokens: ['the', 'co', '##vid', '-', '19', 'vaccine', 'is', 'safe', 'and', 'effective', '.']
Token IDs: [101, 1996, 2522, 17258, 1011, 2539, 17404, 2003, 3647, 1998, 4621, 1012, 102]
